In [2]:
#This Code predict Label for SVF Issues 

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsSVF_20102019.csv')

y = list(data['Label']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:80])
y_train = np.asarray(y_enc[:80])


x_test = np.asarray(x_enc[80:])
y_test = np.asarray(y_enc[80:])

Using TensorFlow backend.


In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [4]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=30, batch_size=32)
    model.save_weights('./model.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30



80/80 [==============================] - 3s 43ms/step - loss: 2.6638 - acc: 0.2750
Epoch 2/30
80/80 [==============================] - 1s 11ms/step - loss: 2.5128 - acc: 0.5250
Epoch 3/30
80/80 [==============================] - 1s 11ms/step - loss: 2.3482 - acc: 0.5125
Epoch 4/30
80/80 [==============================] - 1s 11ms/step - loss: 2.1862 - acc: 0.5125
Epoch 5/30
80/80 [==============================] - 1s 10ms/step - loss: 2.0155 - acc: 0.5125
Epoch 6/30
80/80 [==============================] - 1s 10ms/step - loss: 1.8617 - acc: 0.5125
Epoch 7/30
80/80 [==============================] - 1s 10ms/step - loss: 1.7532 - acc: 0.5125
Epoch 8/30
80/80 [==============================] - 1s 11ms/step - loss: 1.6939 - acc: 0.5125
Epoch 9/30
80/80 [==============================] - 1s 10ms/step - loss: 1.6490 - acc: 0.5125
Epoch 10/30
80/80 [==============================] - 1s 11ms/step - loss: 1.6168 - acc: 0.5125
Epoch 11/30
80/80 [==============================] - 1s 10ms/step - lo

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [7]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))

                         precision    recall  f1-score   support

                    bug       0.00      0.00      0.00         1
            enhancement       0.50      1.00      0.67         1
enhancement|help wanted       0.00      0.00      0.00         2
    help wanted|invalid       0.00      0.00      0.00         1
   help wanted|question       0.00      0.00      0.00         2
               question       0.50      1.00      0.67         5

               accuracy                           0.50        12
              macro avg       0.17      0.33      0.22        12
           weighted avg       0.25      0.50      0.33        12



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# New Section